In [3]:
import pandas as pd
from pandas import DataFrame as df
import sqlite3 as sq
import pprint
import csv

In [4]:
data = pd.read_csv('135-201 (2).csv')

In [5]:
# 需要把所有已经匹配分类的专业index放到列表，再检查没有匹配的专业，手动分类？

<font color=#900088 size=6 face="黑体">高考数据库读取</font>

In [9]:
conn = sq.connect('高考数据库.sqlite3')     # 连接数据库
c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table'")     # 读取表名
table = c.fetchall()
cates = table[10][0]     # 获取类名
mmajors = table[14][0]     # 获取专业表名
colleges = table[15][0]     # 获取学校名
province = table[12][0]
rankings = table[13][0]
fir = table[11][0]

# 获取表的列名、列名结构
def col(form):
    c.execute("SELECT * FROM {}".format(form))
    col_name = [tuple[0] for tuple in c.description]
    
    c.execute("PRAGMA table_info({})".format(form))
    col_structure = c.fetchall()
    
    return col_name,col_structure

# 获取表的内容（DF结构）
def df(form):
    sql = "SELECT * FROM {}".format(form)
    values = c.execute(sql)
    f_df = pd.DataFrame(data = values)
    return f_df

<font color=#00aaaa size=6 face="黑体">专业分类</font>

In [57]:
# majors数据
major = majors_df[majors_df[2] == 2020][majors_df[9] == 1]
# del major_hn['year']
# del major_hn['c4']
# del major_hn['city']
major
major_hn = major.drop(axis=1,columns=[2,7,9])

major_hn.columns = ['id','majorName','score','c1','c2','c3','collage']
length = major_hn.shape[0]
major_hn

<ipython-input-57-c0e0980f0a93>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  major = majors_df[majors_df[2] == 2020][majors_df[9] == 1]


,id,majorName,score,c1,c2,c3,collage
166652,183679,波斯语,623,None,None,None,10036
166688,183715,阿拉伯语,624,None,None,None,10036
166689,183716,朝鲜语,624,None,None,None,10036
166726,183753,中国语言文学类(汉语国际教育、汉语言文学),625,None,None,None,10036
166727,183754,政治学类(国际政治、政治学与行政学),625,None,None,None,10036
...,...,...,...,...,...,...,...
186461,5528704,野生动物与自然保护区管理（国家专项）,611,None,None,None,10225
186462,5531969,计算机科学与技术(中外合作办学),589,None,None,None,10225
186463,5535235,化学(中外合作办学),577,None,None,None,10225
186464,5538502,生物技术(中外合作办学),575,None,None,None,10225


In [28]:
major_new = pd.DataFrame(columns = ['id','majorName','score','c1','c2','c3','collage']) 

In [29]:
def classification(mo,c,mn):     # 初始表、专业表、最终表
    for i in range(length):
        for cate in c[1]:     # c1
            for word in cate[1]:     # c2
                if word in mo['majorName'][i:i+1].values[0]:
#                     print(i,pd.DataFrame(mo.iloc[i:i+1,]))
                    mn = pd.concat([mn,mo[i:i+1]],axis=0 ,ignore_index=True)
                    end = mn.shape[0]
                    mn['c1'][end-1:end] = c[0]
                    mn['c2'][end-1:end] = cate[0]
#                     mn['c3'][end-1:end] = word
                    break
    return mn

In [30]:
law1 = ['法学类',['法学','知识产权','监狱学','信用风险管理与法律防控','国际经贸规则','司法警察学','社区矫正']]
law2 = ['政治学类',['政治学','行政学','国际政治','外交学','国际事务与国际关系','政治学、经济学与哲学','国际组织与全球治理']]
law3 = ['社会学类',['社会学','社会工作','人类学','女性学','家政学','老年学']]
law4 = ['民族学类',['民族学']]
law5 = ['马克思主义理论类',['科学社会主义','中国共产党历史','思想政治教育','马克思主义理论']]
law6 = ['公安学类',['治安','侦察','边防管理','禁毒','警犬技术','经济犯罪侦查','边防指挥','消防指挥','警卫学','公安情报学','犯罪学','公安管理学','涉外警务','国内安全保卫','警务指挥与战术','技术侦查','海景执法','公安政治工作','移民管理','出入境管理']]
law = ['法学',[law1,law2,law3,law4,law5,law6]]

eco1 = ['经济学类',['理科试验班类（经济、金融与管理）','经济管理试验班','经济学','经济统计学','国民经济管理学','资源与环境经济学','商务经济学','能源经济学','劳动经济学','经济工程','数字经济']]
eco2 = ['财政学类',['财政学','税收学']]
eco3 = ['金融学类',['金融学','金融工程','保险学','投资学','金融数学','信用管理','经济与金融','精算学','互联网金融','金融科技']]
eco4 = ['经济与贸易类',['经济与贸易','国际经济与贸易','贸易经济']]
economics = ['经济学',[eco1,eco2,eco3,eco4]]

phi = ['哲学',[['哲学类',['哲学','逻辑学','宗教学','伦理学']]]]

peda1 = ['教育学类',['教育学','科学教育','人文教育','教育技术学','艺术教育','学前教育','小学教育','特殊教育','华文教育','教育康复学','卫生教育','认知科学与技术']]
peda2 = ['体育学类',['体育教育','运动训练','社会体育指导与管理','武术与民族传统体育','运动人体科学','运动康复','休闲体育','体能训练','冰雪运动','电子竞技运动与管理','智能体与工程','体育旅游','运动能力开发']]
pedagogy = ['教育学',[peda1,peda2]]

lit1 = ['中国语言文学类',['中国语言文学类','汉语言文学','汉语言','汉语国际教育','中国少数民族语言文学','古典文献学','应用语言学','秘书学','中国语言与文化','手语翻译']]
lit2 = ['外国语言文学类',['外国语言文学类','语言学','翻译','英语','俄语','日语']]
lit3 = ['新闻传播学类',['新闻','广播电视学','广告学','传播学','编辑出版学','网络与新媒体','数字出版','时尚传播','国际新闻与传播']]
literature = ['文学',[lit1,lit2,lit3]]

history = ['历史学',[['历史学类',['历史学','世界史','考古学','文物与博物馆学','文物保护技术','外国语言与外国历史','文化遗产']]]]

agr1 = ['植物生产类',['植物生产类','农学','园艺','植物保护','植物科学与技术','种子科学与工程','设施农业科学与工程','茶学','烟草','应用生物科学','农艺教育','园艺教育','智慧农业','菌物科学与工程','农药化肥']]
agr2 = ['自然保护与环境生态类',['农业资源与环境','野生动物与自然保护区管理','水土保持与荒漠化防治','生物质科学与工程']]
agr3 = ['动物生产类',['动物生产类','动物科学','蚕学','蜂学','经济动物学','马业科学']]
agr4 = ['动物医学类',['动物医学','动物药学','动植物检疫','实验动物学','中兽医学']]
agr5 = ['林学类',['林学','园林','森林保护','经济林']]
agr6 = ['水产类',['水产类','水产养殖学','海洋渔业科学与技术','水族科学与技术','水生动物医学']]
agr7 = ['草学类',['草业科学','草坪科学与工程']]
agronomy = ['农学',[agr1,agr2,agr3,agr4,agr5,agr6,agr7]]

med1 = ['基础医学类',['基础医学','生物医学','生物医学科学']]
med2 = ['临床医学类',['临床医学','麻醉学','医学影像学','眼视光医学','精神医学','放射医学','儿科学','医学试验班']]
med3 = ['口腔医学类',['口腔医学']]
med4 = ['公共卫生与预防医学类',['预防医学','食品卫生与营养学','妇幼保健医学','卫生监督','全球健康学']]
med5 = ['中医学类',['中医学','针灸推拿学','藏医学','蒙医学','维医学','壮医学','哈医学','傣医学','回医学','中医康复学','中医养生学','中医儿科学','中医骨伤科学']]
med6 = ['中西医结合类',['中西医临床医学']]
med7 = ['药学类',['药学','药物制剂','临床药学','药事管理','药物分析','药物化学','海洋药学','化妆品科学与技术']]
med8 = ['中药学类',['中药学','中药资源与开发','藏药学','蒙药学','中药制药','中草药栽培与鉴定']]
med9 = ['法医学类',['法医学']]
med10 = ['医学技术类',['医学技术','医学检验技术','医学实验技术','医学影像技术','眼视光学','康复治疗学','口腔医学技术','卫生检验与检疫','听力与言语康复学','康复物理治疗','康复作业治疗','智能医学工程']]
med11 = ['护理学类',['护理学','助产学']]
medicine = ['医学',[med1,med2,med3,med4,med5,med6,med7,med8,med9,med10,med11]]

mana1 = ['管理科学与工程类',['管理科学','信息管理与信息系统','工程管理','房地产开发与管理','工程造价','保密管理','邮政管理','大数据管理与应用','工程审计','计算金融','应急管理']]
mana2 = ['工商管理类',['工商管理','市场营销','会计学','财务管理','国际商务','人力资源管理','审计学','资产评估','物业管理','文化产业管理','劳动关系','体育经济与管理','财务会计教育','市场营销教育','零售业管理']]
mana3 = ['农业经济管理类',['农林经济管理','农村区域发展']]
mana4 = ['公共管理类',['公共管理','公共事业管理','行政管理','劳动与社会保障','土地资源管理','城市管理','海关管理','交通管理','海事管理','公共关系学','健康服务与管理','海警后勤管理','医疗产品管理','医疗保险','养老服务管理']]
mana5 = ['图书情报与档案管理类',['图书情报与档案管理类','图书馆学','档案学','信息资源管理']]
mana6 = ['物流管理与工程类',['物流管理','物流工程','采购管理','供应链管理']]
mana7 = ['工业工程类',['工业工程','标准化工程','质量管理工程']]
mana8 = ['电子商务类',['电子商务','电子商务及法律','跨境电子商务']]
mana9 = ['旅游管理类',['旅游管理','酒店管理','会展经济与管理','旅游管理与服务教育']]
management = ['管理学',[mana1,mana2,mana3,mana4,mana5,mana6,mana7,mana8,mana9]]

art = ['艺术学',[['艺术学',['艺术学理论','艺术','音乐','舞蹈','戏剧','影视','美术']]]]

In [31]:
major_law = classification(major_hn,law,major_new)

major_law = major_law[~major_law['id'].isin([459064])]
major_law = major_law[~major_law['id'].isin([472915])]
major_law = major_law[~major_law['id'].isin([475135])]

In [33]:
major_eco = classification(major_hn,economics,major_new)

major_eco = major_eco[~major_eco['id'].isin([459784])]

In [34]:
major_phi = classification(major_hn,phi,major_new)

major_phi = major_phi[~major_phi['id'].isin([2044384])]

In [37]:
major_peda = classification(major_hn,pedagogy,major_new)

In [36]:
def lit_classify(mo,c,mn):     # 初始表、专业表、最终表
    for i in range(length):
        if (mo['majorName'][i:i+1].values[0])[-1] == '语':
            mn = pd.concat([mn,mo[i:i+1]],axis=0 ,ignore_index=True)
            end = mn.shape[0]
            mn['c1'][end-1:end] = '文学'
            mn['c2'][end-1:end] = '外国语言文学类'
#             mn['c3'][end-1:end] = mo['majorName'][i:i+1].values[0]
        else:
            for cate in c[1]:     # c1
                for word in cate[1]:     # c2
                    if word in mo['majorName'][i:i+1].values[0]:
    #                         print(i,pd.DataFrame(mo.iloc[i:i+1,]))
                        mn = pd.concat([mn,mo[i:i+1]],axis=0 ,ignore_index=True)
                        end = mn.shape[0]
                        mn['c1'][end-1:end] = c[0]
                        mn['c2'][end-1:end] = cate[0]
#                         mn['c3'][end-1:end] = word
                        break
    
        for ii in ['人文科学试验班','文科试验班','社会科学试验班']:
            if ii in mo['majorName'][i:i+1].values[0]:
                mn = pd.concat([mn,mo[i:i+1]],axis=0 ,ignore_index=True)
                end = mn.shape[0]
                mn['c1'][end-1:end] = '文学'
        
    return mn

In [38]:
major_lit = lit_classify(major_hn,literature,major_new)

major_lit = major_lit[~major_lit['id'].isin([1962727])]
major_lit = major_lit[~major_lit['id'].isin([5150455])]
major_lit = major_lit[~major_lit['id'].isin([1991002])]
major_lit = major_lit[~major_lit['id'].isin([2106352])]
major_lit = major_lit[~major_lit['id'].isin([408559])]
major_lit = major_lit[~major_lit['id'].isin([423002])]

In [39]:
major_hist = classification(major_hn,history,major_new)

major_hist = major_hist[~major_hist['id'].isin([2044384])]
major_hist = major_hist[~major_hist['id'].isin([4680745])]
major_hist = major_hist[~major_hist['id'].isin([4680745])]

In [40]:
major_agr = classification(major_hn,agronomy,major_new)

In [41]:
major_med = classification(major_hn,medicine,major_new)

major_med = major_med[~major_med['id'].isin([5277115])]

In [42]:
major_mana = classification(major_hn,management,major_new)

major_mana = major_mana[~major_mana['id'].isin([459064])]
major_mana = major_mana[~major_mana['id'].isin([4612159])]
major_mana = major_mana[~major_mana['id'].isin([4686734])]
major_mana = major_mana[~major_mana['id'].isin([4701724])]
major_mana = major_mana[~major_mana['id'].isin([475135])]
major_mana = major_mana[~major_mana['id'].isin([4511740])]

In [43]:
major_art = classification(major_hn,art,major_new)

In [44]:
# 专业去重
all_m = [major_law,major_eco,major_phi,major_peda,major_lit,major_hist,major_agr,major_mana,major_med,major_art]

In [47]:
major = major_new
for i in all_m:
    major = pd.concat([major,i],axis=0 ,ignore_index=True)

In [48]:
major

,id,majorName,score,c1,c2,c3,collage
0,183754,政治学类(国际政治、政治学与行政学),625,法学,政治学类,None,10036
1,183942,法学类(法学),630,法学,法学类,None,10036
2,184909,法学类(法学),653,法学,法学类,None,10036
3,200230,法学,602,法学,法学类,None,10270
4,200377,社会工作,599,法学,社会学类,None,10270
...,...,...,...,...,...,...,...
1627,4852999,播音与主持艺术,546,艺术学,艺术学,None,10200
1628,4856050,美术学,532,艺术学,艺术学,None,10200
1629,4865209,播音与主持艺术,556,艺术学,艺术学,None,10200
1630,4868264,美术学,430,艺术学,艺术学,None,10200


In [870]:
# major.to_csv('major(1).csv',encoding = 'utf-8-sig')

In [1366]:
def same_m(l,m):
    for i in l:
        print(i,list(m[m['id'] == i] ['majorName']))
        
def check(ma,mb):
    id_l = []
    for idd in list(ma['id']):
#         print(idd)
        if idd in list(mb['id']):
#             pprint.pprint(ma[ma['id'] == idd])
            id_l.append(idd)
#     print(id_l)
    same_m(id_l,ma)
    return id_l

In [974]:
for i in range(len(all_m)):
    for j in range(i+1,len(all_m)):
        print(all_m[i]['c1'][0],all_m[j]['c1'][0])
        check(all_m[i],all_m[j])

法学 经济学
1998580 ['政治学、经济学与哲学']
法学 哲学
1998580 ['政治学、经济学与哲学']
法学 教育学
法学 文学
法学 历史学
法学 农学
法学 管理学
法学 医学
220727 ['法医学（法医学与法学双学士学位）']
法学 艺术学
经济学 哲学
1998580 ['政治学、经济学与哲学']
经济学 教育学
经济学 文学
经济学 历史学
经济学 农学
经济学 管理学
经济学 医学
经济学 艺术学
哲学 教育学
哲学 文学
哲学 历史学
哲学 农学
哲学 管理学
哲学 医学
哲学 艺术学
教育学 文学
教育学 历史学
教育学 农学
教育学 管理学
教育学 医学
教育学 艺术学
文学 历史学
文学 农学
文学 管理学
文学 医学
文学 艺术学
历史学 农学
历史学 管理学
历史学 医学
历史学 艺术学
农学 管理学
农学 医学
5277115 ['动物药学']
农学 艺术学
管理学 医学
管理学 艺术学
医学 艺术学


In [796]:
left.to_csv('major_left2.csv',encoding = 'utf-8-sig')

In [797]:
import pandas as pd
lleft = pd.read_csv('major_left_left.csv',encoding='utf-8')

In [798]:
lleft

,Unnamed: 0,id,majorName,score,c1,c2,c3,collage
0,2,200239,公共管理类,600,NaN,NaN,NaN,10270
1,33,202570,医学技术类（医学四年类）,636,NaN,NaN,NaN,10533
2,34,202780,中国语言文学类,627,NaN,NaN,NaN,10533
3,35,202852,外国语言文学类,627,NaN,NaN,NaN,10533
4,36,202925,all,614,NaN,NaN,NaN,10559
...,...,...,...,...,...,...,...,...
75,930,1460302,公共管理类,666,NaN,NaN,NaN,10001
76,931,1461890,中国语言文学类,665,NaN,NaN,NaN,10001
77,934,1469845,外国语言文学类,645,NaN,NaN,NaN,10004
78,941,1484227,经济管理试验班,647,NaN,NaN,NaN,10004


In [829]:
for i in all_m:
    for j in list(i['id']):
        lleft = lleft[~lleft['id'].isin([j])]

In [830]:
lleft

,Unnamed: 0,id,majorName,score,c1,c2,c3,collage
4,36,202925,all,614,NaN,NaN,NaN,10559
5,37,202999,all,636,NaN,NaN,NaN,10559
7,88,210377,轻工类,659,NaN,NaN,NaN,10561
11,120,221545,all,618,NaN,NaN,NaN,10593
12,121,221752,all,600,NaN,NaN,NaN,10593
14,164,239564,高校专项,571,NaN,NaN,NaN,10542
15,166,242129,高校专项,568,NaN,NaN,NaN,10542


<font color=#8800cc size=6 face="黑体">学校和城市对应</font>

In [54]:
loca = pd.read_csv('colleges地域精确到市.csv',encoding = 'utf-8')
major_last = pd.read_csv('major_last.csv',encoding = 'utf-8',index_col = 0)

major_last['college_loca'] = ''
for j in major_last['collegeID_id']:
    for k in id:
#         print(c,k)
        if j == k:
#             print(loca[loca['collegeID'] == k]['city'])
            major_last.loc[major_last['collegeID_id'] == j,'college_loca'] = loca[loca['collegeID'] == k]['city'].values[0]
            break
        
# major_last.to_csv('major_last(2).csv',encoding = 'utf-8-sig')


,id,majorName,year,minScore,categoryID_id,collegeID_id,c1,c2
0,183679,波斯语,2020,623,1,10036,文学,外国语言文学类
1,183715,阿拉伯语,2020,624,1,10036,文学,外国语言文学类
2,183716,朝鲜语,2020,624,1,10036,文学,外国语言文学类
3,183753,中国语言文学类(汉语国际教育、汉语言文学),2020,625,1,10036,文学,中国语言文学类
4,183754,政治学类(国际政治、政治学与行政学),2020,625,1,10036,法学,政治学类
...,...,...,...,...,...,...,...,...
3291,5528704,野生动物与自然保护区管理（国家专项）,2020,611,2,10225,农学,自然保护与环境生态类
3292,5531969,计算机科学与技术(中外合作办学),2020,589,2,10225,工学,计算机类
3293,5535235,化学(中外合作办学),2020,577,2,10225,理学,化学类
3294,5538502,生物技术(中外合作办学),2020,575,2,10225,理学,生物科学类


<font color=#8800cc size=5 face="黑体">gx_jn专业分类</font>

In [50]:
addi = pd.read_csv('gx_jn.csv',encoding = 'utf-8')

d = {
    'id':range(0,44),
    'majorName':addi['major'],
    'score':addi['score'],
    'c1':None,
    'c2':None,
    'c3':None,
    'college':addi['college'],
    'college_loca':None
    }
major_addi = pd.DataFrame(d)
major_addi

,id,majorName,score,c1,c2,c3,college,college_loca
0,0,计算机类,622,None,None,None,10593,None
1,1,数学类,621,None,None,None,10593,None
2,2,机械类,621,None,None,None,10593,None
3,3,土木工程,620,None,None,None,10593,None
4,4,经济学类,620,None,None,None,10593,None
5,5,材料类,619,None,None,None,10593,None
6,6,工商管理类,619,None,None,None,10593,None
7,7,化学工程与工艺,618,None,None,None,10593,None
8,8,矿物资源工程,618,None,None,None,10593,None
9,9,轻工类,618,None,None,None,10593,None


In [51]:
def classification2(mo,c,mn):     # 初始表、专业表、最终表
    for i in range(44):
        if c[0] in mo['majorName'][i:i+1].values[0]:     # c1
            mn = pd.concat([mn,mo[i:i+1]],axis=0 ,ignore_index=True)
            end = mn.shape[0]
            mn['c1'][end-1:end] = c[0]
            for cate in c[1]:     # c2
                if cate[0] in mo['majorName'][i:i+1].values[0]:
                    mn['c2'][end-1:end] = cate[0]
                else:
                    for word in cate[1]:     # c2
                        if word in mo['majorName'][i:i+1].values[0]:
                            mn['c2'][end-1:end] = cate[0]
        #                     mn['c3'][end-1:end] = word
                            break
        else:
            for cate in c[1]:     # c2
                if cate[0] in mo['majorName'][i:i+1].values[0]:
                    mn = pd.concat([mn,mo[i:i+1]],axis=0 ,ignore_index=True)
                    end = mn.shape[0]
                    mn['c1'][end-1:end] = c[0]
                    mn['c2'][end-1:end] = cate[0]
                else:
                    for word in cate[1]:     # c2
                        if word in mo['majorName'][i:i+1].values[0]:
                            mn = pd.concat([mn,mo[i:i+1]],axis=0 ,ignore_index=True)
                            end = mn.shape[0]
                            mn['c1'][end-1:end] = c[0]
                            mn['c2'][end-1:end] = cate[0]
        #                     mn['c3'][end-1:end] = word
                            break
    return mn

In [55]:
# major_last[0:3]
for i in major_addi['majorName']:
    for j in major_last['majorName']:
        if i == j:
#             print(major_last[major_last['majorName'] == j]['c1'].values[0])
#             print(i,j)
            major_addi.loc[major_addi['majorName'] == i,'c1'] = major_last[major_last['majorName'] == j]['c1'].values[0]
            major_addi.loc[major_addi['majorName'] == i,'c2'] = major_last[major_last['majorName'] == j]['c2'].values[0]
            major_addi.loc[major_addi['majorName'] == i,'category'] = str(major_last[major_last['majorName'] == j]['categoryID_id'].values[0])
            break

major_addi.loc[major_addi['majorName'] == '矿物资源工程','c1'] = '工学'
major_addi.loc[major_addi['majorName'] == '矿物资源工程','c2'] = '矿业类'
major_addi

In [58]:
d = {
#     'Unnamed:0':range(len(major_addi['collage'])),
    'id':range(0,44),
    'majorName':major_addi['majorName'],
    'year':'2020',
    'minScore':major_addi['score'],
    'categoryID_id':major_addi['category'],
    'collegeID_id':major_addi['college'],
    'c1':major_addi['c1'],
    'c2':major_addi['c2'],
    'college_loca':None,
    }

adddi = pd.DataFrame(d)
adddi.loc[adddi['collegeID_id'] == 10593,'collegeID_id'] = '10593'
adddi.loc[adddi['collegeID_id'] == 10559,'collegeID_id'] = '10559'
adddi.loc[adddi['collegeID_id'] == '10593','college_loca'] = '广西省南宁市'
adddi.loc[adddi['collegeID_id'] == '10559','college_loca'] = '广东省广州市'

nnew = pd.concat([major_last,adddi],axis=0 ,ignore_index=True)
nnew

,id,majorName,year,minScore,categoryID_id,collegeID_id,c1,c2,college_loca
0,183679,波斯语,2020,623,1,10036,文学,外国语言文学类,NaN
1,183715,阿拉伯语,2020,624,1,10036,文学,外国语言文学类,NaN
2,183716,朝鲜语,2020,624,1,10036,文学,外国语言文学类,NaN
3,183753,中国语言文学类(汉语国际教育、汉语言文学),2020,625,1,10036,文学,中国语言文学类,NaN
4,183754,政治学类(国际政治、政治学与行政学),2020,625,1,10036,法学,政治学类,NaN
...,...,...,...,...,...,...,...,...,...
3335,39,法学,2020,618,1,10559,法学,法学类,广东省广州市
3336,40,电子商务,2020,618,2,10559,管理学,电子商务类,广东省广州市
3337,41,酒店管理,2020,617,1,10559,管理学,旅游管理类,广东省广州市
3338,42,历史学,2020,614,1,10559,历史学,历史学类,广东省广州市
